In [1]:
#  STEP 1: Install and Import Required Libraries
# ------------------------------------------------
# 🔧 Keep Colab's default NumPy (2.2.x) and pandas (2.2.2) to avoid binary mismatch.
# Prophet 1.1.5 works fine with them if cmdstanpy is reinstalled cleanly.
!pip install --force-reinstall prophet==1.1.5 cmdstanpy==1.2.0 --quiet

import numpy as np

# ✅ Compatibility patch for old Prophet references
if not hasattr(np, "float_"):
    np.float_ = np.float64

import pandas as pd
from prophet import Prophet
import os

print("✅ Prophet and dependencies loaded successfully!")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.4 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.4 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompat

In [2]:
# 🪜 STEP 2: Load and Explore the Dataset
# ------------------------------------------------
from google.colab import files

# Upload dataset manually each session
files.upload()  # 👈 Select your 'mock_kaggle.csv' file when prompted

# Load dataset
df = pd.read_csv("/content/mock_kaggle.csv")

# Rename columns from Portuguese → English
df.rename(columns={
    'data': 'date',
    'venda': 'sales',
    'estoque': 'stock',
    'preco': 'price'
}, inplace=True)

# Display dataset info
print("🔹 First 5 rows:")
display(df.head())

print("\n🔹 Dataset Info:")
print(df.info())

print("\n🔹 Missing Values:")
print(df.isnull().sum())

print("\n🔹 Summary Statistics:")
display(df.describe())


Saving mock_kaggle.csv to mock_kaggle.csv
🔹 First 5 rows:


,date,sales,stock,price
0,2014-01-01,0,4972,1.29
1,2014-01-02,70,4902,1.29
2,2014-01-03,59,4843,1.29
3,2014-01-04,93,4750,1.29
4,2014-01-05,96,4654,1.29



🔹 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    937 non-null    object 
 1   sales   937 non-null    int64  
 2   stock   937 non-null    int64  
 3   price   937 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 29.4+ KB
None

🔹 Missing Values:
date     0
sales    0
stock    0
price    0
dtype: int64

🔹 Summary Statistics:


,sales,stock,price
count,937.000000,937.000000,937.000000
mean,90.533618,1608.258271,1.592572
std,80.682089,1356.691877,0.529502
min,0.000000,0.000000,0.000000
25%,33.000000,794.000000,1.290000
50%,76.000000,1348.000000,1.390000
75%,127.000000,1964.000000,1.890000
max,542.000000,7228.000000,2.980000


In [3]:
#  STEP 3: Clean and Structure the Data
# ------------------------------------------------
# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Sort by date
df = df.sort_values('date')

# Drop missing 'date' or 'sales'
df = df.dropna(subset=['date', 'sales'])

# Keep relevant columns
df = df[['date', 'sales']]

# Preview cleaned data
print("\n✅ Cleaned dataset preview:")
display(df.head())

print(f"\n📆 Date Range: {df['date'].min()} → {df['date'].max()}")



✅ Cleaned dataset preview:


,date,sales
0,2014-01-01,0
1,2014-01-02,70
2,2014-01-03,59
3,2014-01-04,93
4,2014-01-05,96



📆 Date Range: 2014-01-01 00:00:00 → 2016-07-31 00:00:00


In [4]:
# 🪜 STEP 4: Feature Engineering
# ------------------------------------------------
# Extract time-based features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.strftime('%b')

# Monthly average sales
monthly_avg = df.groupby(['year', 'month'])['sales'].mean().reset_index()
monthly_avg.rename(columns={'sales': 'monthly_avg_sales'}, inplace=True)

# Merge back to main DataFrame
df = pd.merge(df, monthly_avg, on=['year', 'month'], how='left')

# Mark December as holiday season
df['is_holiday_season'] = df['month'].apply(lambda x: 1 if x == 12 else 0)

print("✅ Feature engineering complete. Sample data:")
display(df.head())

print("\n🔹 Columns after feature engineering:")
print(df.columns.tolist())


✅ Feature engineering complete. Sample data:


,date,sales,year,month,month_name,monthly_avg_sales,is_holiday_season
0,2014-01-01,0,2014,1,Jan,132.833333,0
1,2014-01-02,70,2014,1,Jan,132.833333,0
2,2014-01-03,59,2014,1,Jan,132.833333,0
3,2014-01-04,93,2014,1,Jan,132.833333,0
4,2014-01-05,96,2014,1,Jan,132.833333,0



🔹 Columns after feature engineering:
['date', 'sales', 'year', 'month', 'month_name', 'monthly_avg_sales', 'is_holiday_season']


In [5]:
# 🪜 STEP 5: Train the Forecasting Model (Prophet)
# ------------------------------------------------
# Prophet requires 'ds' and 'y' columns
df_prophet = df[['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})

# Initialize and fit model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

model.fit(df_prophet)

# Create future dates (next 90 days)
future = model.make_future_dataframe(periods=90)

# Predict future sales
forecast = model.predict(future)

print("✅ Forecast generated successfully!")
display(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())


DEBUG:cmdstanpy:input tempfile: /tmp/tmphds7wti3/uo5z8gb3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphds7wti3/ognm49h4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=45871', 'data', 'file=/tmp/tmphds7wti3/uo5z8gb3.json', 'init=/tmp/tmphds7wti3/ognm49h4.json', 'output', 'file=/tmp/tmphds7wti3/prophet_modellyux2yc8/prophet_model-20251029134728.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:47:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:47:28 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


✅ Forecast generated successfully!


,ds,yhat,yhat_lower,yhat_upper
1022,2016-10-25,224.754129,130.517312,314.177578
1023,2016-10-26,228.126650,125.990946,311.950883
1024,2016-10-27,231.242215,140.283146,324.374388
1025,2016-10-28,234.014885,148.559112,327.158891
1026,2016-10-29,236.361456,149.627568,330.737999


In [6]:
# 🪜 STEP 6: Merge Actual + Forecast Data
# ------------------------------------------------
forecast_export = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
forecast_export = forecast_export.rename(columns={
    'ds': 'date',
    'yhat': 'predicted_sales',
    'yhat_lower': 'lower_bound',
    'yhat_upper': 'upper_bound'
})

# Merge with original sales
merged_data = pd.merge(df, forecast_export, on='date', how='outer')

print("✅ Actual + Forecast data merged:")
display(merged_data.tail())


✅ Actual + Forecast data merged:


,date,sales,year,month,month_name,monthly_avg_sales,is_holiday_season,predicted_sales,lower_bound,upper_bound
1022,2016-10-25,NaN,NaN,NaN,NaN,NaN,NaN,224.754129,130.517312,314.177578
1023,2016-10-26,NaN,NaN,NaN,NaN,NaN,NaN,228.126650,125.990946,311.950883
1024,2016-10-27,NaN,NaN,NaN,NaN,NaN,NaN,231.242215,140.283146,324.374388
1025,2016-10-28,NaN,NaN,NaN,NaN,NaN,NaN,234.014885,148.559112,327.158891
1026,2016-10-29,NaN,NaN,NaN,NaN,NaN,NaN,236.361456,149.627568,330.737999


from matplotlib import pyplot as plt
_df_0['predicted_sales'].plot(kind='hist', bins=20, title='predicted_sales')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['lower_bound'].plot(kind='hist', bins=20, title='lower_bound')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['upper_bound'].plot(kind='hist', bins=20, title='upper_bound')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['date']
  ys = series['lower_bound']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('lower_bound')

from matplotlib import pyplot as plt
_df_9['predicted_sales'].plot(kind='line', figsize=(8, 4), title='predicted_sales')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['lower_bound'].plot(kind='line', figsize=(8, 4), title='lower_bound')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['upper_bound'].plot(kind='line', figsize=(8, 4), title='upper_bound')
plt.gca().spines[['top', 'right']].set_visible(False)

In [7]:
# 🪜 STEP 7: Export for Power BI Dashboard
# ------------------------------------------------
output_path = "/content/retail_sales_forecast.csv"
merged_data.to_csv(output_path, index=False)

print(f"✅ Forecast data exported successfully! File saved at: {output_path}")

# Optional: download to your system
from google.colab import files
files.download(output_path)


✅ Forecast data exported successfully! File saved at: /content/retail_sales_forecast.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>